In [70]:
import torch
import torch.nn as nn
import torch.optim as optim

import cv2 as cv
import mediapipe as mp

import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from IPython.display import display, Image
from sklearn.preprocessing import MinMaxScaler, Normalizer

from tqdm import tqdm
import os
import sys

import json
from glob import glob
from collections import OrderedDict

# Define the Neural Network

In [71]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        # First fully connected layer
        self.fc1 = nn.Linear(input_size, hidden_size)  
        # Activation function
        self.relu = nn.ReLU()
        # Output layer
        self.fc2 = nn.Linear(hidden_size, num_classes)  

    def forward(self, x):
        # Forward pass: compute predicted y
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define model parameters
input_size = 63
hidden_size = 128  # You can adjust this value
num_classes = 24

# Instantiate the model
model = NeuralNet(input_size, hidden_size, num_classes)

# Define Loss Function and Optimizer

In [72]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # lr is the learning rate

# Training the Model

In [73]:
def train(model, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, target)
            
            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

# You would call this function with your data loader
# train(model, your_data_loader, epochs=10)

# Evaluation

In [74]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Call this with your test data loader
# test(model, your_test_data_loader)

# test

In [75]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pandas import DataFrame

In [76]:
df = pd.read_json("processed_data.json")
# df['label'] = df['label'].replace({'A': 0, 'B': 1})
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z12,z13,z14,z15,z16,z17,z18,z19,z20,label
0,0.174138,0.203343,0.229114,0.245728,0.246139,0.236828,0.235357,0.228395,0.228840,0.213760,...,0.231463,0.248762,0.233060,0.236895,0.240368,0.222757,0.217457,0.222375,0.230399,A
1,0.154908,0.199391,0.247105,0.268197,0.270157,0.253825,0.249148,0.234938,0.229938,0.222525,...,0.230862,0.253146,0.231245,0.239760,0.248545,0.227390,0.216762,0.227295,0.241564,A
2,0.159652,0.191880,0.228607,0.257717,0.252847,0.257458,0.247691,0.231889,0.229773,0.232824,...,0.230438,0.250651,0.238889,0.240883,0.241257,0.225991,0.223449,0.227114,0.233759,A
3,0.164240,0.201327,0.234502,0.259028,0.267355,0.246959,0.244995,0.234087,0.236546,0.217758,...,0.231807,0.248318,0.233374,0.239115,0.242942,0.223314,0.217289,0.224675,0.234197,A
4,0.151370,0.195552,0.242214,0.266695,0.271002,0.249770,0.250443,0.233929,0.226981,0.219453,...,0.232041,0.251699,0.231547,0.239487,0.247135,0.224706,0.215417,0.225201,0.238368,A


In [77]:
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder

# # Assuming df is your DataFrame
# le = LabelEncoder()

# # Fit and transform your DataFrame's last column
# df['label'] = le.fit_transform(df.iloc[:, -1])

In [78]:
# df.head()

In [79]:
# from sklearn.model_selection import train_test_split

# # Split the data into features (X) and labels (y)
# X = df.iloc[:, :-1]  # First 63 columns as features
# y = df['label']  # Last column as the label

# # Split the dataset into training set and test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
# import torch
# from torch.utils.data import TensorDataset, DataLoader

# # Convert to tensors
# train_tensors = TensorDataset(torch.tensor(X_train.values, dtype=torch.float), torch.tensor(y_train.values, dtype=torch.long))
# test_tensors = TensorDataset(torch.tensor(X_test.values, dtype=torch.float), torch.tensor(y_test.values, dtype=torch.long))

# # Create DataLoaders
# batch_size = 32  # Choose according to your needs or performance considerations
# train_loader = DataLoader(train_tensors, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_tensors, batch_size=batch_size, shuffle=False)

In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch

# Assuming your DataFrame is named 'df'
# Convert output strings to integers
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# Split features and target
X = df.iloc[:, :63].values  # Features
y = df['label'].values  # Target

# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to float32 for better compatibility with PyTorch
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [82]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y).long()  # Long tensor for classification labels

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [83]:
# Create datasets
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Define data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [84]:
train(model, train_loader, 40)
test(model, test_loader)

Train Epoch: 0 [0/6612 (0%)]	Loss: 3.180721
Train Epoch: 0 [3200/6612 (48%)]	Loss: 3.147403
Train Epoch: 0 [6400/6612 (97%)]	Loss: 2.978036
Train Epoch: 1 [0/6612 (0%)]	Loss: 2.932304
Train Epoch: 1 [3200/6612 (48%)]	Loss: 2.796500
Train Epoch: 1 [6400/6612 (97%)]	Loss: 2.452566
Train Epoch: 2 [0/6612 (0%)]	Loss: 2.489521
Train Epoch: 2 [3200/6612 (48%)]	Loss: 2.392346
Train Epoch: 2 [6400/6612 (97%)]	Loss: 2.207166
Train Epoch: 3 [0/6612 (0%)]	Loss: 2.013390
Train Epoch: 3 [3200/6612 (48%)]	Loss: 1.849828
Train Epoch: 3 [6400/6612 (97%)]	Loss: 1.876013
Train Epoch: 4 [0/6612 (0%)]	Loss: 1.891037
Train Epoch: 4 [3200/6612 (48%)]	Loss: 1.749223
Train Epoch: 4 [6400/6612 (97%)]	Loss: 1.522879
Train Epoch: 5 [0/6612 (0%)]	Loss: 1.310601
Train Epoch: 5 [3200/6612 (48%)]	Loss: 1.491680
Train Epoch: 5 [6400/6612 (97%)]	Loss: 1.492205
Train Epoch: 6 [0/6612 (0%)]	Loss: 1.482730
Train Epoch: 6 [3200/6612 (48%)]	Loss: 1.011822
Train Epoch: 6 [6400/6612 (97%)]	Loss: 1.216321
Train Epoch: 7 [0/66

In [95]:
# df.iloc[0].tolist()[:-1]

import numpy as np

np.argmax(model(torch.Tensor(df.iloc[0].tolist()[:-1])).detach().numpy())
# model(torch.Tensor(df.iloc[0].tolist()[:-1]))

0

In [109]:
inp = pd.read_json('test.json').iloc[0].to_list()
np.argmax(model(torch.Tensor(inp)).detach().numpy())


9